In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")
from sklearn.model_selection import cross_val_score
#leer carpetas
import os

# Base de datos

In [2]:
# folder path
dir_fold = 'C:/Users/carlo/OneDrive - Universidad Nacional de Colombia/Semestre 11/MachineLearning/DownScaling/VariablesListas'

# list to store Folders
folder = []

# Iterate directory
for path in os.listdir(dir_fold):
    # check if current path is a file
    #if os.path.isfile(os.path.join(dir_path, path)):
        folder.append(path)

orden = [3,2,6,5,1,0,4]

df = pd.DataFrame()

for i in orden:
    df_i = pd.read_csv(dir_fold+'/'+folder[i])
    df = pd.concat([df, df_i], axis=1)

X = df.drop(['Ppt_SIATA'], axis=1)
y = df['Ppt_SIATA']

df.head(2)

,Ppt_ERA5,Humedad,Temperatura,Presion,Elevación,CoorX,CoorY,Ppt_SIATA
0,5.47598,33.672039,13.844298,802.956631,1781.0,-75.28,6.55,43.023724
1,5.47598,33.672039,14.142333,803.052634,1624.0,-75.27,6.55,38.443913


# 20: Ensambles de métodos

Los dos métodos de ensamblajes mas populares son:

**Bagging**. Entrena y acopla múltiples modelos, generalmente del mismo tipo, de forma paralela e independiente con diferentes conjuntos de muestras de entrenamiento.

**Boosting**. Entrena y acopla múltiples modelos, generalmente del mismo tipo, de forma secuencial y donde cada modelo individual aprende del error del modelo previo.

## *Bagging*
Agregacion tipo *Bagging* consiste en acoplar múltiples modelos tomando conjunto de muestras de entrenamiento aleatorias, con reemplazamiento. El resultado final es un promedio o moda de todas las predicciones de los submodelos. Los modelos tipos *Bagging* mas conocidos son: (i) *Bagged Decision Trees*, (ii) *Random Forest*.

## Regresión

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.95, random_state=1)

In [4]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
print(rf.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


La función *pprint* entrega de forma organizada la información:

In [5]:
from pprint import pprint
pprint(rf.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [6]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [8]:
# search across 50 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 50, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=2)

In [9]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 50,
 'bootstrap': False}

In [10]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [5, 10, 15],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [1000, 1500, 1800, 2000]
}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                           scoring = 'neg_mean_absolute_error', cv = 3, 
                           n_jobs = -1, verbose = 2)

In [11]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [5, 10, 15],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [3, 5, 7],
                         'n_estimators': [1000, 1500, 1800, 2000]},
             scoring='neg_mean_absolute_error', verbose=2)

In [12]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 15,
 'min_samples_leaf': 3,
 'min_samples_split': 3,
 'n_estimators': 2000}

In [15]:
rf_best1 = RandomForestRegressor(n_estimators=1800,min_samples_split=5,min_samples_leaf=4,max_features='sqrt',max_depth=10,bootstrap=False)
rf_best1.fit(X_train, y_train);
y_pred = rf_best1.predict(X_test)
print(rf_best1.score(X_test,y_test))

0.21817229489167955


In [25]:
y_pred_vect = rf_best1.predict(X)

In [30]:
np.mean(y_pred_vect)

2.8053898621908138

In [26]:
#Guardar Precipitación SIATA
(pd.DataFrame(zip(y_pred_vect),columns=['Ppt_EnsambleBagging'])).to_csv('Resultados_Ppt/Ppt_EnsambleBagging.csv', index=False)

In [14]:
rf_best2 = RandomForestRegressor(bootstrap=True, max_depth=10, min_samples_leaf=5, min_samples_split=7, n_estimators=1000)
rf_best2.fit(X_train, y_train);
y_pred = rf_best2.predict(X_test)
print(rf_best2.score(X_test,y_test))

0.20921916791393091


## Boosting Algorithms
Los algortimso de ensamble crean una secuencia de modelos que aprenden de los errores de los modelos previos. Posteriormente las predicciones las realiza el modelo sopesado. Los dos algoritmos tipo *Boosting* son: (i) AdaBoost, y (ii) Stochastic Gradient Boosting.

### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
adb = AdaBoostClassifier(n_estimators=30, random_state=1)

In [ ]:
adb.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=30, random_state=1)

In [ ]:
y_pred = adb.predict(X_test)
print(adb.score(X_test,y_test))
print(classification_report(y_test,y_pred))

0.9666666666666667
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      0.92      0.96        13
           2       0.86      1.00      0.92         6

    accuracy                           0.97        30
   macro avg       0.95      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30



In [ ]:
importances = list(adb.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(iris.feature_names, importances)]
feature_importances

[('sepal length (cm)', 0.0),
 ('sepal width (cm)', 0.0),
 ('petal length (cm)', 0.5),
 ('petal width (cm)', 0.5)]

In [ ]:
results = cross_val_score(adb, X, y, cv=kfold)
print(results.mean())

0.9400000000000001


### Stochastic Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=100, random_state=1)

In [ ]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=1, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
y_pred = gbc.predict(X_test)
print(gbc.score(X_test,y_test))
print(classification_report(y_test,y_pred))

0.9666666666666667
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      0.92      0.96        13
           2       0.86      1.00      0.92         6

    accuracy                           0.97        30
   macro avg       0.95      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30



In [ ]:
results = cross_val_score(model, X, y, cv=kfold)
print(results.mean())

In [ ]:
importances = list(gbc.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(iris.feature_names, importances)]
feature_importances

## XGBoost

In [ ]:
import xgboost as xgb 
model = xgb.XGBClassifier(objective='binary:logistic', n_estimators= 7, seed=44)

In [ ]:
model.fit(X_train, y_train) 

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=7, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=44, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=44, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [ ]:
y_pred = model.predict(X_test)
accuracy = float(np.sum(y_pred == y_test)) / y_test.shape[0]
accuracy

0.058823529411764705